# Learning

In [1]:
import numpy as np
from pickle import load

In [3]:
features = load(open("features4.pickle", "rb"))
y_all = load(open("manner.pickle", "rb"))

In [4]:
y_all.shape

(2631171,)

In [52]:
from sklearn.cross_validation import train_test_split

selected = np.random.randint(y_all.shape[0],size=10000)
X = features[selected,:]
y = y_all[selected]

X, X_test, y, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [53]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X = enc.fit_transform(X)
X_test = enc.transform(X_test)

# Guess cause of death

## SVC

In [54]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import SVC
#from sklearn import cross_validation

# X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, y, test_size=0.33, random_state=5122)

#gamma = 0.1
#C = 10
#clf = OneVsRestClassifier(SVC(kernel='rbf', C=C, gamma=gamma, probability=True), n_jobs=4)

#clf.fit(X, y)
#print(clf.score(X_test, y_test))

## K neighbors

In [55]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=300)
print("TRAINING....")
clf.fit(X, y)
print("SCORE:")
print(clf.score(X_test, y_test))

TRAINING....
SCORE:
0.784242424242


# Naive Bayes

In [56]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
print("TRAINING....")
clf.fit(X.toarray(), y)
print("SCORE:")
print(clf.score(X_test.toarray(), y_test))

TRAINING....
SCORE:
0.784242424242


Black man with no education

In [57]:
ex = enc.transform([[1, 0, 3, 0]])
clf.predict_proba(ex)

array([[ 0.10967773,  0.20974399,  0.01057556,  0.10739365,  0.00292666,
         0.05527228,  0.50441013]])

White woman married, with high education

In [58]:
ex = enc.transform([[6, 1, 1, 1]])
clf.predict_proba(ex)

array([[  1.76107116e-01,   1.80795828e-02,   3.79939991e-03,
          4.09118148e-04,   1.39970385e-04,   7.25061514e-04,
          8.00739751e-01]])

In [59]:
ex = enc.transform([[6, 1, 1, 0]])
clf.predict_proba(ex)

array([[ 0.17466828,  0.07274633,  0.01398445,  0.00306439,  0.00142965,
         0.00462858,  0.72947833]])